El GCPD (Gotham City Police Dept) recolecta la información de casos policiales que acontecen en Ciudad Gótica. Esta información se encuentra guardada en un dataframe con el siguiente formato: (fecha, id_caso, descripcion, estado_caso, categoria, latitud, longitud).  

Los  posibles estados  que puede tener  un caso son 1: caso abierto, 2: caso  resuelto, 3: cerrado sin resolución.  

Las fechas se encuentran en el formato YYYY-MM-DD. Por otro lado el comisionado Gordon guarda un registro detallado sobre en cuáles casos fue activada la batiseñal para 
pedir ayuda del vigilante, Batman. Esta información se encuentra en un Dataframe con el siguiente formato (id_caso, respuesta), siendo campo respuesta si la señal tuvo una respu
esta positiva (1) o negativa (0) de parte de él.

El  sector  encargado  de  las  estadísticas  oficiales  del  GCPD  quiere  con  esta  información  analizar  las  siguientes situaciones: 

a) Tasa de resolución de casos de la fuerza policial por categoría de caso (considerando aquellos casos en los que no participó Batman). 

b) Tasa de resolución de casos con la ayuda de Batman (considerando que aquellos casos en los que fue llamado con la batiseñal, participó en la resolución). 

c) Indicar el mes del año pasado en el que Batman tuvo mayor participación en la investigación de casos. 

[Link](https://piazza.com/class_profile/get_resource/jkr2voxi1yw4wt/jkr2vqu7n114zx)

**Resolución de colaborador de la materia: [Link](https://github.com/juanlaura/Datos/blob/master/Pandas/Parcial_1C_2018/Parcial%202018%201C.ipynb)**

In [1]:
# Set-up y vista rápida de los dos datasets sintéticos

CASO_ABIERTO = 1
CASO_RESUELTO = 2
CASO_CERRADO = 3

BATMAN_NO_PARTICIPA = 0
BATMAN_PARTICIPA = 1

import pandas as pd

df_gcpd = pd.read_csv('../data/2018C1_GCPD.csv')
display(df_gcpd)

df_gordon = pd.read_csv('../data/2018C1_gordon.csv')
display(df_gordon)

,fecha,id_caso,descripcion,estado_caso,categoria,latitud,longitud
0,2016-05-03,1,Mataron a uno,1,Arseny,100,100
1,2016-05-03,2,Mataron al Guason!!,2,Robo,100,100
2,2017-07-03,3,Incendio,1,Robo,100,100
3,2017-07-03,4,Robaron un banco,3,Incendio,100,100
4,2017-08-03,5,Mataron a uno,2,Incendio,100,100
5,2017-08-03,6,Mataron a uno,1,Arseny,100,100
6,2017-10-03,7,Mataron a uno,2,Robo,100,100
7,2017-10-03,8,Mataron a uno,3,Incendio,100,100
8,2017-11-03,9,Mataron a uno,2,Arseny,100,100
9,2017-12-03,10,Mataron a uno,1,Robo,100,10


,id_caso,respuesta
0,1,1
1,2,0
2,3,0
3,4,0
4,5,1
5,6,0
6,7,1
7,8,0
8,9,1
9,10,0


**a) Tasa de resolución de casos de la fuerza policial por categoría de caso (considerando aquellos casos en los que no participó Batman).**

Los pasos a seguir serían:
- Filtrar los casos donde batman no participó
- Unir con el dataframe de información de casos
- Agrupar por ´categoria´
- Calcular el % de casos resueltos de cada categoria

In [2]:
df_sin_batman = df_gordon[df_gordon['respuesta'] == BATMAN_NO_PARTICIPA]
df = df_gcpd.merge(df_sin_batman, how='left', on='id_caso')
df.groupby(['categoria'])['estado_caso'].agg(lambda x: (x == CASO_RESUELTO).mean()).rename('tasa de resolución')

categoria
Arseny      0.333333
Incendio    0.333333
Robo        0.500000
Name: tasa de resolución, dtype: float64

**b) Tasa de resolución de casos con la ayuda de Batman (considerando que aquellos casos en los que fue llamado con la batiseñal, participó en la resolución)**

Los pasos son muy similares a los del anterior paso, pero filtrando los casos donde Batman participó y sin agrupar por categoría

In [3]:
df_con_batman = df_gordon[df_gordon['respuesta'] == BATMAN_PARTICIPA]
df = df_gcpd.merge(df_con_batman, how='inner',on='id_caso')
df.estado_caso.apply(lambda x: x==CASO_RESUELTO).mean()

0.75

**c) Indicar el mes del año pasado en el que Batman tuvo mayor participación en la investigación de casos.**

- Vamos a filtrar por participación de Batman
- Vamos a unir los dataframes por ´id_caso´
- Vamos a filtrar los casos por el año 2017
- Tomar solo el mes de cada caso
- Calcular el/los valores mas repetidos

In [4]:
df_con_batman = df_gordon.loc[df_gordon['respuesta']==BATMAN_PARTICIPA] 
df = df_gcpd.merge(df_con_batman, how='inner',on='id_caso') 
df[df.fecha.str.startswith("2017")].fecha.apply(lambda x: x[5:7]).mode().values.tolist()

['08', '10', '11']